In [1]:
import re
import os
import sys
import nltk
import xml.etree.cElementTree as et
import pickle
import base64
from collections import *

In [2]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1
# print(stop_words)

In [3]:
pattern = re.compile("[^a-zA-Z]")

title_inverted_index = defaultdict(list)
body_inverted_index = defaultdict(list)
category_inverted_index = defaultdict(list)
infobox_inverted_index = defaultdict(list)

In [4]:
# arguments = sys.argv
# wikipedia_dump = arguments[0]
wikipedia_dump = "/home/vatsal/Documents/IIIT/Sem-3/IRE/wikipedia/wiki.xml"
content = et.iterparse(wikipedia_dump, events=("start", "end"))
content = iter(content)
# document_title = open("index/document_title.pickle", "wb")

In [5]:
document_no = 0
title_freq = {}
body_freq = {}
category_freq = {}
infobox_freq = {}
document_title = {}
document_word = {}

In [6]:
for event,context in content:
    tag = re.sub(r"{.*}", "", context.tag)
    
    if event == "start":
        if tag == "page":
            title_freq = {}
            body_freq = {}
            category_freq = {}
            infobox_freq = {}
    
    elif event == "end":
        
        if tag == "title":
            
            title_text = context.text
            document_title[document_no]=title_text
            title_text = title_text.lower()
            words = re.split(pattern, title_text)
            for word in words:
                word = stemmer.stem(word)
                if word and word not in stop_words:
                    if word not in title_freq:
                        title_freq[word] = 1
                    else:
                        title_freq[word] += 1
        
        elif tag == "text":
            
            body_text = context.text
            try:
                category_words = re.findall("\[\[Category:(.*?)\]\]", body_text);
                if category_words:
                    for category_word in category_words:
                        words = re.split(pattern, category_word)
                        for word in words:
                            word = word.lower();
                            word = stemmer.stem(word)
                            if  word and word not in stop_words:
                                if word not in category_freq:
                                    category_freq[word] = 1
                                else:
                                    category_freq[word] += 1
            except:
                pass
            
            try:
                info_words = re.findall("{{Infobox(.*?)}}", body_text)
                if info_words :
                    for info_word in info_words:
                        words = re.split(pattern, info_word)
                        for word in words:
                            word = word.lower()
                            word = stemmer.stem(word)
                            if word and word not in stop_words:
                                if word not in infobox_freq:
                                    infobox_freq[word] = 1
                                else:
                                    infobox_freq[word] += 1
            except:
                pass
                                    
            try:
                words = re.split(pattern, body_text)

                for word in words:
                    word = word.lower()
                    word = stemmer.stem(word)
                    if word and word not in stop_words:
                        if word not in body_freq:
                            body_freq[word] = 1
                        else:
                            body_freq[word] += 1
            except:
                pass
            
        elif tag == "page":
            index = str(document_no)
            for word in body_freq:
                s = index + ":" + str(body_freq[word])
                body_inverted_index[word].append(s)

            for word in title_freq:
                s = index + ":" + str(title_freq[word])
                title_inverted_index[word].append(s)

            for word in category_freq:
                s = index + ":" + str(category_freq[word])
                category_inverted_index[word].append(s)

            for word in infobox_freq:
                s = index + ":" + str(infobox_freq[word])
                infobox_inverted_index[word].append(s)
                
            document_no += 1
        context.clear()

In [7]:
pointer = 0
filename = "index/title.txt"
title_file = open(filename, "w+")
for word in sorted(title_inverted_index) :
    index = ",".join(title_inverted_index[word])
    index = word + "-" + index + "\n"
    title_file.write(index)
    document_word[word] = {}
    document_word[word]['t'] = pointer
    pointer += len(index)
title_file.close()

In [8]:
pointer = 0
filename = "index/category.txt"
category_file = open(filename, "w+")
for word in sorted(category_inverted_index) :
    index = ",".join(category_inverted_index[word])
    index = word + "-" + index + "\n"
    category_file.write(index)
    if word not in document_word:
        document_word[word] = {}
    document_word[word]['c'] = pointer
    pointer += len(index)
category_file.close()

In [9]:
pointer = 0
filename = "index/infobox.txt"
infobox_file = open(filename, "w+")
for word in sorted(infobox_inverted_index) :
    index = ",".join(infobox_inverted_index[word])
    index = word + "-" + index + "\n"
    infobox_file.write(index)
    if word not in document_word:
        document_word[word] = {}
    document_word[word]['i'] = pointer
    pointer += len(index)
infobox_file.close()

In [10]:
pointer = 0
filename = "index/body_text.txt"
body_text_file = open(filename, "w+")
for word in sorted(body_inverted_index) :
    index = ",".join(body_inverted_index[word])
    index = word + "-" + index + "\n"
    body_text_file.write(index)
    if word not in document_word:
        document_word[word] = {}
    document_word[word]['b'] = pointer
    pointer += len(index)
body_text_file.close()

In [11]:
file = open("index/title_doc_no.pickle", "wb")
pickle.dump(document_title, file)
file.close()

In [12]:
file = open("index/word_position.pickle", "wb")
pickle.dump(document_word, file)
file.close()